In [1]:
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langgraph.graph import StateGraph, END

In [2]:
df = pd.read_csv("..\data\processed\Articulos_LLM.csv")

In [3]:
docs = [
    Document(
        page_content=row["contenido"],
        metadata={
            "titulo": row["titulo"],
            "url": row["url"],
            "precio": row.get("Precios", ""),
            "fecha": row.get("fechas", ""),
            "contexto_fecha":row.get("fechas_contexto",""),
            "edad": row.get("edad", ""),
        }
    )
    for _, row in df.iterrows()
]


In [4]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " "]
)

# Crear los Document chunked
chunked_docs = []

for _, row in df.iterrows():
    texto = row["contenido"]
    if pd.isnull(texto):
        continue
    chunks = splitter.split_text(texto)
    for i, chunk in enumerate(chunks):
        chunked_docs.append(Document(
            page_content=chunk,
            metadata={
            "titulo": row["titulo"],
            "url": row["url"],
            "precio": row.get("Precios", ""),
            "fecha": row.get("fechas", ""),
            "contexto_fecha":row.get("fechas_contexto",""),
            "edad": row.get("edad", ""),
        }
    )
        )

In [5]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Vectorstore
vectorstore = FAISS.from_documents(chunked_docs, embedding_model)

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

# Modelo local desde Hugging Face
# model_name = "microsoft/Phi-3-mini-4k-instruct"
llm = HuggingFaceEndpoint(
    repo_id="google/flan-t5-large",
    task="text2text-generation",
    huggingfacehub_api_token="",
    max_new_tokens=512,
    do_sample=False
)

# chat_model = ChatHuggingFace(llm=llm)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [7]:
# from langchain.llms import HuggingFacePipeline
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# # Carga local del modelo
# model_id = "HuggingFaceH4/zephyr-7b-beta"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)

# # Crear pipeline
# hf_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

# # Integrar con LangChain
# from langchain_core.language_models import LLM
# from langchain.llms import HuggingFacePipeline

# llm = HuggingFacePipeline(pipeline=hf_pipe)


In [8]:
# from typing import TypedDict, List

# class AgentState(TypedDict):
#     query: str
#     documents: List[Document]
#     generation: str

# def retrieve(state):
#     query = state["query"]
#     docs = vectorstore.max_marginal_relevance_search(query, k=3, fetch_k=50)
#     return {"documents": docs}

# def generate(state):
#     docs = state["documents"]
#     query = state["query"]

#     context = "\n\n".join(doc.page_content for doc in docs)

#     prompt = f"""
# Contesta exclusivamente a la pregunta usando la información proporcionada. 
# No inventes detalles ni enlaces si no aparecen en el contexto. 
# Si no hay suficiente información, di claramente que no puedes responder.

# Contexto:
# {context}

# Pregunta: {query}

# Respuesta informativa y honesta:
# """

#     # Obtener texto desde el modelo local
#     respuesta = llm(prompt)

#     # Extraer enlaces de metadata
#     enlaces = {
#         doc.metadata.get("url"): doc.metadata.get("titulo", "Sin título")
#         for doc in docs if doc.metadata.get("url")
#     }
#     links_texto = "\n\nEnlaces relacionados:\n" + "\n".join(f"{titulo} → {url}" for url, titulo in enlaces.items())

#     return {
#         "generation": f"{respuesta.strip()}{links_texto}"
#     }


In [7]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
from langchain.schema import Document

# Estado del agente
class AgentState(TypedDict):
    query: str
    documents: List[Document]
    generation: str

# Paso retrieval
def retrieve(state):
    query = state["query"]
    docs = vectorstore.max_marginal_relevance_search(query, k=3, fetch_k=50)
    return {"documents": docs}

# Paso generación usando contexto
def generate(state):
    docs = state["documents"]
    query = state["query"]

    context = "\n\n".join(doc.page_content for doc in docs)

    prompt = f"""
Contesta exclusivamente a la pregunta usando la información proporcionada. 
No inventes detalles ni enlaces si no aparecen en el contexto. 
Si no hay suficiente información, di claramente que no puedes responder.

Contexto:
{context}

Pregunta: {query}

Respuesta informativa y honesta:
"""

    respuesta = llm.invoke(prompt)

    enlaces = {
        doc.metadata.get("url"): doc.metadata.get("titulo", "Sin título")
        for doc in docs if doc.metadata.get("url")
    }
    links_texto = "\n\nEnlaces relacionados:\n" + "\n".join(f"{titulo} → {url}" for url, titulo in enlaces.items())

    return {
        "generation": f"{respuesta.strip()}{links_texto}"
    }



# Grafo del agente
graph = StateGraph(AgentState)
graph.add_node("retrieve", retrieve)
graph.add_node("generate", generate)
graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "generate")
graph.add_edge("generate", END)

# Compilar
rag_agent = graph.compile()

In [ ]:
# graph = StateGraph(AgentState)
# graph.add_node("retrieve", retrieve)
# graph.add_node("generate", generate)
# graph.set_entry_point("retrieve")
# graph.add_edge("retrieve", "generate")
# graph.add_edge("generate", END)

# # Compilar
# rag_agent = graph.compile()


In [8]:
respuesta = rag_agent.invoke({"query": "¿Qué actividades hay este fin de semana?"})
print(respuesta["generation"])

c:\Users\Abdon\Documents\GitHub\LLM_Desafio\.venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


ValueError: Model 'google/flan-t5-large' doesn't support task 'text2text-generation'.